# An extension of the project for DBSCAN, OPTICS and such


In [ ]:
## using DBSCN to visualize crime
from sklearn.cluster import DBSCAN
import sklearn.utils
from sklearn.preprocessing import StandardScaler
sklearn.utils.check_random_state(1000)
Clus_dataSet = arrestloc[['Latitude','Longitude']]
Clus_dataSet = np.nan_to_num(Clus_dataSet)
Clus_dataSet = StandardScaler().fit_transform(Clus_dataSet)

# Compute DBSCAN #eps=.1, min =750,|||eps=.075, min_samples=200
# best cluster: db = DBSCAN(eps=.075, min_samples=150).fit(Clus_dataSet)
db = DBSCAN(eps=.075, min_samples=150).fit(Clus_dataSet)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
arrestloc["Clus_Db"]=labels

realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
clusterNum = len(set(labels)) 

arrestloc.Clus_Db.unique()

# Basemap

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = (14,10)
llon=-74.210
ulon=-73.618
llat=40.55
ulat=42
m = Basemap(projection='merc',llcrnrlat=40.55,urcrnrlat=40.9,\
            llcrnrlon=-74.1, urcrnrlon=-73.82, resolution='f', area_thresh = 0.01)



m.drawmapboundary(fill_color='#85A6D9')
m.drawcoastlines(color='#6D5F47', linewidth=.4)
m.drawrivers(color='#6D5F47', linewidth=.2)
m.fillcontinents(color='white',lake_color='#85A6D9')
m.shadedrelief()

xs,ys = m(np.asarray(arrestloc.Longitude), np.asarray(arrestloc.Latitude))
arrestloc['xm']= xs.tolist()
arrestloc['ym'] =ys.tolist()

colors = plt.get_cmap('jet')(np.linspace(0.0, 1.0, clusterNum))
    
    
for index,row in arrestloc.iterrows():
    m.plot(row.xm, row.ym, markerfacecolor =([1,0,0]),  marker='o', markersize= 5, alpha = 0.75)

# Folium Looping

In [ ]:
#Single Cluster
map_ny = folium.Map(location=[latitude, longitude], zoom_start=10)
colormap = {0.0: 'pink', 0.3: 'blue', 0.5: 'green',  0.7: 'yellow', 1.0: 'red'}
# A sample of clusters
clus_8=arrestloc[arrestloc['Clus_Db']==5]
clus_8
# plotting coordinates of postal code, with stopname as labels in popup
for lat, lng, crime in zip(clus_8.Latitude.to_list(), clus_8.Longitude.to_list(), clus_8.OFNS_DESC.to_list()):
    label = 'Crime:{}'.format(crime)
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=True).add_to(map_ny)  
    
map_ny

In [ ]:
#ALL CLUSTERS-WARNING THIS IS A BITCH TO RUN
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(-1,36)
ys = [i + x + (i*x)**2 for i in range(36)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(arrestloc['Latitude'], arrestloc['Longitude'], arrestloc['Clus_Db']):
    label = folium.Popup(str(' Cluster ' + str(cluster)))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)